# Derive Top N Portfolio

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

In [2]:
import os
import sys
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from utils.modules.etl.load import sparkDBwls as sdb
from utils.modules.etl.transform import sparkCleanNRich as scne
from wrangler.modules.assets.etl import dataPrep as prep
# from utils.modules.ml.timeseries import rollingstats as stats

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    sdb = importlib.reload(sdb)
    scne = importlib.reload(scne)
    prep = importlib.reload(prep)
#     stats= importlib.reload(stats)
    
__desc__ = "analyze crypto market capitalization time series data"
# clsSDB = sdb.SQLWorkLoads(desc=__desc__)
clsSCNR=scne.Transformer(desc=__desc__)
# clsStat=stats.RollingStats(desc=__desc__)
clsPrep =prep.RateOfReturns(desc=__desc__)
''' optional - if not specified class will use the default values '''
# prop_kwargs = {"WRITE_TO_TMP":True,   # necessary to emulate the etl dag
#               }
print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
All functional SPARKCLEANNRICH-libraries in TRANSFORM-package of ETL-module imported successfully!
All functional DATAPREP-libraries in ETL-package of ASSETS-module imported successfully!
All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
All functional SPARKCLEANNRICH-libraries in TRANSFORM-package of ETL-module imported successfully!
All functional DATAPREP-libraries in ETL-package of ASSETS-module imported successfully!
All functional SPARKNOSQLWLS-libraries in LOAD-package of ETL-module imported successfully!
sparkNoSQLwls Class initialization complete
dataPrep Class initialization complete

Class initialization and load complete!


## Read data from mcap_past
We apply a query to select assets with mcap > 1.0 million. Any missing values are imputed with the mean value.

In [3]:
_from_date = '2022-01-01'
_to_date = '2022-01-31'
# _query = "select * from warehouse.mcap_past "+\
#         f"where mcap_date >= '{_from_date}' and "+\
#         f"mcap_date <= '{_to_date}'"
_query = "select * from warehouse.mcap_past wmp "+\
        f"where wmp.mcap_date between '{_from_date}' and '{_to_date}' "+\
        f"and wmp.mcap_value > 10000 "
_kwargs = {
    "TABLENAME":'warehouse.mcap_past',
    "COLUMN":'mcap_date',
    "FROMDATETIME":_from_date,
    "TODATETIME":_to_date,
    "PARTITIONS":2,
    "AGGREGATE":'avg',
    "LOGBASE":'10',
    "PIVCOLUMNS":['dxd','sofi','wsn','xmx','uqc','btr','unic','nex','noia',
                  'hanu','aca','bbs','xvs','pnd','shake','stpl','dtx','tethys',
                  'kyoko','boba','nlife','rare','eved','yfl','fkx','flixx',
                  'drk','meto','glide','shr','tetu','mft','cmerge','shmn','tronpad']
}

# print(clsSpark.dbSchema)
mcap_sdf = clsPrep.read_n_clean_mcap(query=_query,**_kwargs)
# mcap_sdf = clsROR.read_n_clean_mcap(**_kwargs)

print("Loaded %d rows and %d columns" % (mcap_sdf.count(),len(mcap_sdf.columns)))

Wait a moment, retrieving data ...
23/02/21 14:32:22 WARN Utils: Your hostname, FarmRaiderTester resolves to a loopback address: 127.0.1.1; using 192.168.124.15 instead (on interface enp2s0)
23/02/21 14:32:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/21 14:32:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/21 14:32:24 WARN FileSystem: Cannot load filesystem: java.util.ServiceConfigurationError: org.apache.hadoop.fs.FileSystem: com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem Unable to get public no-arg constructor
23/02/21 14:32:24 WARN FileSystem: java.lang.NoClassDefFoundError: com/google/api/client/auth/oauth2/Credential
23/02/21 14:32:24 WARN FileSystem: java.lang.ClassNotFoundException: com.google.api.client.auth.oauth2.Credential


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/21 14:32:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


23/02/21 14:32:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Loaded 74300 rows and 17 columns


In [10]:
from pyspark.sql import functions as F

mcap_sdf=mcap_sdf.sort(F.col('mcap_date'),F.col('mcap_value'))
mcap_sdf=mcap_sdf.filter(F.col('mcap_past_pk').isNotNull())
mcap_sdf.select(F.col('mcap_past_pk'),F.col('mcap_date'),F.col('asset_name'),
                F.col('mcap_value'),F.col('log_ror'))\
    .filter(F.col('log_ror').isNotNull())\
    .show(n=5)

+------------+-------------------+----------+--------------------+------------------+
|mcap_past_pk|          mcap_date|asset_name|          mcap_value|           log_ror|
+------------+-------------------+----------+--------------------+------------------+
|       49195|2022-01-02 00:00:00|       sas|10065.83106084450...|0.6666558637706426|
|      169511|2022-01-02 00:00:00|      edao|11296.84312717820...|0.7089117639589353|
|       39375|2022-01-02 00:00:00|      scho|12291.45593138100...|0.6943239878505439|
|       17824|2022-01-02 00:00:00|     rigel|12864.00343692410...|0.6865061778350110|
|      165997|2022-01-02 00:00:00|     distx|14332.52767831190...|0.6844287855620943|
+------------+-------------------+----------+--------------------+------------------+
only showing top 5 rows



## Compute LogROR for all assets

In [11]:
_ror='SIMP'
_ror_col=None

if _ror=='NATLOG':
    _ror_col='log_ror'
elif _ror=='SIMP':
    _ror_col='simp_ror'
else:
    pass
_kwargs={
    "PREVALCOLNAME":'mcap_lag',
    "DIFFCOLNAME":'mcap_diff',
    "RORCOLNAME":_ror_col,
}

_mcap_log_ror, _ror_col = clsPrep.get_ror(
    data=mcap_sdf,
    ror_type=_ror,
    num_col ="mcap_value",
    part_col='asset_name',
    date_col='mcap_date',
    **_kwargs,
)

## Write ROR data to DB

In [17]:
_upsert_sdf=_mcap_log_ror.select( 
    F.col('mcap_past_pk'),
    F.col('asset_name'),
    F.col('mcap_date'),
    F.col('mcap_value'),
    F.col(_ror_col),
)\
    .filter((F.col(_ror_col).isNotNull()))
_upsert_sdf.show(n=5)

_records=clsPrep.write_data_to_db(
    data=_upsert_sdf,
#     **kwargs,
)
print("Upserted %d records" % _records)

+------------+----------+-------------------+--------------------+----------+
|mcap_past_pk|asset_name|          mcap_date|          mcap_value|  simp_ror|
+------------+----------+-------------------+--------------------+----------+
|       91384|      btcs|2022-01-02 00:00:00|2785102.132352710...|-17.429416|
|      202976|     sngls|2022-01-02 00:00:00|155996.7475973480...|-15.209829|
|      120739|      shdw|2022-01-02 00:00:00|364515.8216135790...| -2.810053|
|       67523|      sybc|2022-01-02 00:00:00|79187.09948750640...| -1.119790|
|      153610|     scriv|2022-01-02 00:00:00|122004.6214330040...| -1.056263|
+------------+----------+-------------------+--------------------+----------+
only showing top 5 rows

Validating upsert attributes and parameters ...


Wait a moment, writing data to postgresql tip database ...


Upserted 71034 records
